In [1]:
# this is to allow you to demo how some of the functions work between RDS postgresql and python and df work
import sys
import psycopg2
sys.path.append('../')
from helper import db 

In [14]:
# for selecting data
query = """SELECT * from temp_data limit 10"""
db.run_query(query)

,id,date,station,longitude,latitude,depth,fish_type,fish_count,region
0,1,2019-01-01,1,None,None,None,None,None,None
1,2,2018-01-01,1,None,None,None,None,None,None


In [4]:
# for inserting a  single row of data
query = """INSERT INTO temp_data (date, station) VALUES ('2019-01-01', 1)
"""
db.run_query(query)

""


In [13]:
# for inserting dataframes!!!! default uses 'append' unless you specify if_exists differently
import pandas as pd
test = {'date': ['2018-01-01'], 'station': [1]}
df = pd.DataFrame.from_dict(test)
df['date'] = pd.to_datetime(df['date'])
db.insert_table(table_name='temp_data', df=df)

In [60]:
# for inserting on dataframes 

import zipfile
import pandas as pd
zip = zipfile.ZipFile('../data/Teleost_Especes_Pelagiques_Abondance_2009-2018.zip')
zip.namelist()

# let's get the fish mappings - requires specific encoding to be read
FISH_MAP = pd.read_csv('../data/DataDictionary_DictionnaireDonnees.csv', encoding='cp437', skiprows=10).set_index('Species')
FISH_MAP = FISH_MAP[['Name _EN']].to_dict('dict')['Name _EN']
north_fish, south_fish = pd.read_csv(zip.open(zip.namelist()[0])), pd.read_csv(zip.open(zip.namelist()[1]))
north_fish['region'] = 'NORTH'
south_fish['region'] = 'SOUTH'

# let's just append the two datasets together and rename the fish mappings to something slightly more human readable 
fish_set = pd.concat([north_fish, south_fish]).rename(columns=FISH_MAP).copy()
fish_set['Date'] = pd.to_datetime(fish_set['Date'])
fish_set.columns = [i.lower() for i in fish_set.columns]
DATABASE_COLUMN_MAPPING = {'profondeur/depth': 'depth', 'sand lances (ammodytes sp.)': 'sand_lances', 'white barracudina': 'white_barracudina', 
                          'atlantic argentine': 'atlantic_argentine', 'arctic cod': 'arctic_cod', 'atlantic herring': 'atlantic_herring', 
                          'threespine stickleback': 'threespine_stickleback', 'capelin (m. villosus)': 'capelin', 'atlantic soft pout': 'atlantic_soft_pout',
                          'silver hake': 'silver_hake', 'rainbow smelt': 'rainbow_smelt', 'atlantic mackerel (s. scombrus)': 'atlantic_mackerel'}
fish_set.rename(columns=DATABASE_COLUMN_MAPPING, inplace=True)

melted_columns = [i for i in fish_set.columns if i not in ['date', 'station', 'longitude', 'latitude', 'depth', 'region']]
melted_fish_set = pd.melt(fish_set, id_vars=['date', 'station', 'longitude', 'latitude', 'depth', 'region'], value_vars=melted_columns)
melted_fish_set.rename(columns={'value': 'fish_count', 'variable': 'fish_type'}, inplace=True)
display(melted_fish_set)
db.insert_table(table_name='fish_data', df=melted_fish_set)

,date,station,longitude,latitude,depth,region,fish_type,fish_count
0,2009-08-01,8,-59.652417,47.724500,435.0,NORTH,sand_lances,0.0
1,2009-08-01,9,-59.851500,47.833167,502.0,NORTH,sand_lances,0.0
2,2009-08-01,10,-60.046167,47.752250,513.5,NORTH,sand_lances,0.0
3,2009-08-01,11,-60.635833,47.847167,308.5,NORTH,sand_lances,0.0
4,2009-08-02,12,-60.795750,47.998917,383.5,NORTH,sand_lances,0.0
...,...,...,...,...,...,...,...,...
41140,2018-10-01,105,-61.127333,47.353500,48.0,SOUTH,atlantic_mackerel,3.0
41141,2018-10-01,106,-61.023667,47.513167,41.0,SOUTH,atlantic_mackerel,0.0
41142,2018-10-01,107,-60.665000,47.427167,62.0,SOUTH,atlantic_mackerel,0.0
41143,2018-10-01,108,-60.641833,47.230500,106.0,SOUTH,atlantic_mackerel,0.0


In [2]:
# skeleton query for creating the fish table
fish_query = """
CREATE TABLE fish_data (
id SERIAL NOT NULL,
date date NOT NULL,
station int,
longitude decimal (30,10),
latitude decimal (30,10),
depth real,
fish_type varchar(50),
fish_count real,
region varchar(20),
PRIMARY KEY (id)
)
"""

dtspp_query = """
CREATE TABLE dtspp (
id SERIAL NOT NULL, 
measure_time 
)
"""
db.run_query(query)

""


In [19]:
# sample query for deleting a row from the table
query = """
delete from fish_data where id=1
"""
# db.run_query(query)

In [50]:
query = """
DROP TABLE fish_data"""
db.run_query(query)

""
